In [2]:
from google.cloud import bigquery
import pandas as pd 
import chime   
import seaborn as sns
client = bigquery.Client("gcp-wow-rwds-ai-checkout-dev")

/opt/homebrew/lib/python3.11/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)

In [17]:
check = client.query(
"""     
with step_one AS (
    SELECT booking_number, line_name, ARRAY_AGG(DISTINCT sku ORDER BY sku) AS sku_array
    FROM `gcp-wow-cart-data-prod-d710.cdm.dim_cartology_campaigns`, 
    UNNEST(SPLIT(quoteline_sku, ",")) AS sku 
    WHERE opportunity_owner='Alex Moses' 
    AND media_start_date >= "2024-08-01"
    GROUP BY ALL
),
step_two AS (
    SELECT *, ARRAY_TO_STRING(sku_array, ",") AS skus_string 
    FROM step_one 
),
unique_skus AS (
    SELECT DISTINCT skus_string FROM step_two
),
unnested_unique_skus AS (
    SELECT skus_string, sku
    FROM unique_skus, 
    UNNEST(SPLIT(skus_string, ",")) sku 
),
weekly_sales_per_store AS (
    SELECT 
        skus_string, 
        Site, 
        DATE_TRUNC(TXNStartDate, WEEK) AS sales_week,
        COUNT(DISTINCT BasketKey) AS n_transactions
    FROM `gcp-wow-ent-im-wowx-cust-prod.adp_wowx_dm_integrated_sales_view.article_sales_summary_v`sales
    INNER JOIN unnested_unique_skus 
        ON unnested_unique_skus.sku = sales.Article 
    WHERE LOWER(SalesChannelDescription) <> "online"
    AND SalesOrg = 1005
    AND TXNStartDate >= DATE("2024-03-01")
    GROUP BY 1,2,3
),
weekly_averages AS (
    SELECT 
        skus_string, 
        Site, 
        AVG(n_transactions) AS weekly_average_baskets 
    FROM weekly_sales_per_store 
    GROUP BY 1,2
), 
statistics AS (
    SELECT
        skus_string,
        AVG(weekly_average_baskets) AS mean_transactions,
        STDDEV(weekly_average_baskets) AS stddev_transactions
    FROM weekly_averages
    GROUP BY skus_string
),
sample_size_calculation AS (
    SELECT
        skus_string,
        stddev_transactions,
        1.96 AS z_alpha,  -- Z-value for 95% confidence
        0.84 AS z_beta,   -- Z-value for 80% power
        0.01 * AVG(mean_transactions) AS effect_size,  -- Assuming a 10% effect size
        -- Calculate sample size using the simplified formula
        POWER((1.96 + 0.84) * stddev_transactions / (0.1 * AVG(mean_transactions)), 2) AS sample_size_per_group
    FROM statistics
    GROUP BY skus_string, stddev_transactions
),
control_store_recommendation AS (
    SELECT 
        skus_string, 
        CEIL(sample_size_per_group) AS recommended_control_stores
    FROM sample_size_calculation
)
SELECT * FROM control_store_recommendation;
/*
buckets AS (
    SELECT 
        skus_string, 
        Site,
        CASE 
            WHEN weekly_average_baskets = 0 THEN '0'
            WHEN weekly_average_baskets >= 1 AND weekly_average_baskets <= 10 THEN '1 - 10'
            WHEN weekly_average_baskets >= 11 AND weekly_average_baskets <= 20 THEN '11 - 20'
            WHEN weekly_average_baskets >= 21 AND weekly_average_baskets <= 30 THEN '21 - 30'
            WHEN weekly_average_baskets >= 31 AND weekly_average_baskets <= 40 THEN '31 - 40'
            WHEN weekly_average_baskets >= 41 AND weekly_average_baskets <= 50 THEN '41 - 50'
            WHEN weekly_average_baskets >= 51 AND weekly_average_baskets <= 60 THEN '51 - 60'
            WHEN weekly_average_baskets >= 61 AND weekly_average_baskets <= 70 THEN '61 - 70'
            WHEN weekly_average_baskets >= 71 AND weekly_average_baskets <= 80 THEN '71 - 80'
            WHEN weekly_average_baskets >= 81 AND weekly_average_baskets <= 90 THEN '81 - 90'
            WHEN weekly_average_baskets >= 91 AND weekly_average_baskets <= 100 THEN '91 - 100'
            WHEN weekly_average_baskets >= 101 AND weekly_average_baskets <= 110 THEN '101 - 110'
            WHEN weekly_average_baskets >= 111 AND weekly_average_baskets <= 120 THEN '111 - 120'
            WHEN weekly_average_baskets >= 121 AND weekly_average_baskets <= 130 THEN '121 - 130'
            WHEN weekly_average_baskets >= 131 AND weekly_average_baskets <= 140 THEN '131 - 140'
            WHEN weekly_average_baskets >= 141 AND weekly_average_baskets <= 150 THEN '141 - 150'
            ELSE '150+'
        END AS sales_bucket,
        CASE 
            WHEN weekly_average_baskets = 0 THEN 0
            WHEN weekly_average_baskets >= 1 AND weekly_average_baskets <= 10 THEN 1
            WHEN weekly_average_baskets >= 11 AND weekly_average_baskets <= 20 THEN 11
            WHEN weekly_average_baskets >= 21 AND weekly_average_baskets <= 30 THEN 21
            WHEN weekly_average_baskets >= 31 AND weekly_average_baskets <= 40 THEN 31
            WHEN weekly_average_baskets >= 41 AND weekly_average_baskets <= 50 THEN 41
            WHEN weekly_average_baskets >= 51 AND weekly_average_baskets <= 60 THEN 51
            WHEN weekly_average_baskets >= 61 AND weekly_average_baskets <= 70 THEN 61
            WHEN weekly_average_baskets >= 71 AND weekly_average_baskets <= 80 THEN 71
            WHEN weekly_average_baskets >= 81 AND weekly_average_baskets <= 90 THEN 81
            WHEN weekly_average_baskets >= 91 AND weekly_average_baskets <= 100 THEN 91
            WHEN weekly_average_baskets >= 101 AND weekly_average_baskets <= 110 THEN 101
            WHEN weekly_average_baskets >= 111 AND weekly_average_baskets <= 120 THEN 111
            WHEN weekly_average_baskets >= 121 AND weekly_average_baskets <= 130 THEN 121
            WHEN weekly_average_baskets >= 131 AND weekly_average_baskets <= 140 THEN 131
            WHEN weekly_average_baskets >= 141 AND weekly_average_baskets <= 150 THEN 141
            ELSE 151
        END AS sort_key
    FROM weekly_averages 
) 
SELECT 
    skus_string, 
    sales_bucket, 
    sort_key,
    COUNT(DISTINCT Site) AS n_stores 
FROM buckets 
GROUP BY 1,2,3
ORDER BY 1,3
*/







    
"""
).result()
check_df = check.to_dataframe()
check_df


,skus_string,recommended_control_stores
0,"175568,175569,175570,175571,175592,175593,236845,254550,254551,254623,254626,285093,285094,285097,392395",266.0
1,"331595,694123",701.0


In [23]:
check = client.query(
"""     
WITH 
step_one AS (
    SELECT booking_number, line_name, ARRAY_AGG(DISTINCT sku ORDER BY sku) AS sku_array
    FROM `gcp-wow-cart-data-prod-d710.cdm.dim_cartology_campaigns`, 
    UNNEST(SPLIT(quoteline_sku, ",")) AS sku 
    WHERE opportunity_owner='Alex Moses' 
    AND media_start_date >= "2024-08-01"
    GROUP BY booking_number, line_name
),
step_two AS (
    SELECT *, ARRAY_TO_STRING(sku_array, ",") AS skus_string 
    FROM step_one 
),
unique_skus AS (
    SELECT DISTINCT skus_string FROM step_two
),
unnested_unique_skus AS (
    SELECT skus_string, sku
    FROM unique_skus, 
    UNNEST(SPLIT(skus_string, ",")) sku 
),
weekly_sales_per_store AS (
    SELECT 
        skus_string, 
        Site, 
        DATE_TRUNC(TXNStartDate, WEEK) AS sales_week,
        COUNT(DISTINCT BasketKey) AS n_transactions
    FROM `gcp-wow-ent-im-wowx-cust-prod.adp_wowx_dm_integrated_sales_view.article_sales_summary_v`sales
    INNER JOIN unnested_unique_skus 
        ON unnested_unique_skus.sku = sales.Article 
    WHERE LOWER(SalesChannelDescription) <> "online"
    AND SalesOrg = 1005
    AND TXNStartDate >= DATE("2024-03-01")
    GROUP BY skus_string, Site, sales_week
),
weekly_averages AS (
    SELECT 
        skus_string, 
        Site, 
        AVG(n_transactions) AS weekly_average_baskets 
    FROM weekly_sales_per_store 
    GROUP BY skus_string, Site
),
classified_stores AS (
    SELECT 
        skus_string, 
        Site,
        weekly_average_baskets,
        CASE 
            WHEN weekly_average_baskets <= 20 THEN 'Low'
            WHEN weekly_average_baskets BETWEEN 21 AND 60 THEN 'Medium'
            ELSE 'High'
        END AS transaction_group
    FROM weekly_averages 
),
group_counts AS (
    SELECT 
        skus_string, 
        transaction_group, 
        COUNT(*) AS group_count
    FROM classified_stores
    GROUP BY skus_string, transaction_group
),
total_counts AS (
    SELECT 
        skus_string, 
        SUM(group_count) AS total_count
    FROM group_counts
    GROUP BY skus_string
),
proportional_sample_size AS (
    SELECT 
        g.skus_string, 
        g.transaction_group, 
        g.group_count, 
        t.total_count,
        ROUND(g.group_count * 100.0 / t.total_count) AS stratified_sample_size  -- Adjust sample size to 100
    FROM group_counts g
    JOIN total_counts t ON g.skus_string = t.skus_string
),
numbered_stores AS (
    SELECT 
        c.skus_string, 
        c.Site, 
        c.transaction_group,
        c.weekly_average_baskets,
        p.stratified_sample_size,
        ROW_NUMBER() OVER (PARTITION BY c.skus_string, c.transaction_group ORDER BY RAND()) AS row_num
    FROM classified_stores c
    JOIN proportional_sample_size p 
        ON c.skus_string = p.skus_string 
        AND c.transaction_group = p.transaction_group
),
stratified_sample AS (
    SELECT 
        skus_string, 
        Site, 
        transaction_group,
        weekly_average_baskets
    FROM numbered_stores
    WHERE row_num <= stratified_sample_size
),
statistics AS (
    SELECT
        skus_string,
        transaction_group,
        AVG(weekly_average_baskets) AS mean_transactions,
        STDDEV(weekly_average_baskets) AS stddev_transactions
    FROM stratified_sample
    GROUP BY skus_string, transaction_group
),
sample_size_calculation AS (
    SELECT
        skus_string,
        transaction_group,
        stddev_transactions,
        1.96 AS z_alpha,  -- Z-value for 95% confidence
        0.84 AS z_beta,   -- Z-value for 80% power
        AVG(mean_transactions) AS mean_transactions,
        0.1 * AVG(mean_transactions) AS effect_size,  -- Assuming a 10% effect size based on the mean transactions
        -- Calculate sample size using the simplified formula
        POWER((1.96 + 0.84) * stddev_transactions / (0.1 * AVG(mean_transactions)), 2) AS sample_size_per_group
    FROM statistics
    GROUP BY skus_string, transaction_group, stddev_transactions
),
control_store_recommendation AS (
    SELECT 
        skus_string, 
        transaction_group,
        CEIL(sample_size_per_group) AS recommended_control_stores,
        CEIL(sample_size_per_group) AS recommended_test_stores  -- Assuming 1:1 ratio
    FROM sample_size_calculation
)
SELECT * FROM control_store_recommendation;

    
"""
).result()
check_df = check.to_dataframe()
check_df


,skus_string,transaction_group,recommended_control_stores,recommended_test_stores
0,"175568,175569,175570,175571,175592,175593,236845,254550,254551,254623,254626,285093,285094,285097,392395",Low,1.0,1.0
1,"331595,694123",High,387.0,387.0
2,"331595,694123",Medium,81.0,81.0
3,"331595,694123",Low,171.0,171.0
4,"175568,175569,175570,175571,175592,175593,236845,254550,254551,254623,254626,285093,285094,285097,392395",High,120.0,120.0
5,"175568,175569,175570,175571,175592,175593,236845,254550,254551,254623,254626,285093,285094,285097,392395",Medium,44.0,44.0


In [31]:
check = client.query(
"""     
WITH 
step_one AS (
    SELECT booking_number, line_name, ARRAY_AGG(DISTINCT sku ORDER BY sku) AS sku_array
    FROM `gcp-wow-cart-data-prod-d710.cdm.dim_cartology_campaigns`, 
    UNNEST(SPLIT(quoteline_sku, ",")) AS sku 
    WHERE opportunity_owner='Alex Moses' 
    AND media_start_date >= "2024-08-01"
    GROUP BY booking_number, line_name
),
step_two AS (
    SELECT *, ARRAY_TO_STRING(sku_array, ",") AS skus_string 
    FROM step_one 
),
unique_skus AS (
    SELECT DISTINCT skus_string FROM step_two
),
unnested_unique_skus AS (
    SELECT skus_string, sku
    FROM unique_skus, 
    UNNEST(SPLIT(skus_string, ",")) sku 
),
weekly_sales_per_store AS (
    SELECT 
        skus_string, 
        Site, 
        DATE_TRUNC(TXNStartDate, WEEK) AS sales_week,
        COUNT(DISTINCT BasketKey) AS n_transactions
    FROM `gcp-wow-ent-im-wowx-cust-prod.adp_wowx_dm_integrated_sales_view.article_sales_summary_v`sales
    INNER JOIN unnested_unique_skus 
        ON unnested_unique_skus.sku = sales.Article 
    WHERE LOWER(SalesChannelDescription) <> "online"
    AND SalesOrg = 1005
    AND TXNStartDate >= DATE("2024-03-01")
    GROUP BY skus_string, Site, sales_week
),
weekly_averages AS (
    SELECT 
        skus_string, 
        Site, 
        AVG(n_transactions) AS weekly_average_baskets 
    FROM weekly_sales_per_store 
    GROUP BY skus_string, Site
),
classified_stores AS (
    SELECT 
        skus_string, 
        Site,
        weekly_average_baskets,
        CASE 
            WHEN weekly_average_baskets <= 20 THEN 'Low'
            WHEN weekly_average_baskets BETWEEN 21 AND 60 THEN 'Medium'
            ELSE 'High'
        END AS transaction_group,
        CASE 
            WHEN weekly_average_baskets <= 20 THEN '1 - 20'
            WHEN weekly_average_baskets BETWEEN 21 AND 60 THEN '21 - 60'
            ELSE '61+'
        END AS transaction_range
    FROM weekly_averages 
),
group_counts AS (
    SELECT 
        skus_string, 
        transaction_group, 
        transaction_range, 
        COUNT(*) AS group_count
    FROM classified_stores
    GROUP BY skus_string, transaction_group, transaction_range
),
total_counts AS (
    SELECT 
        skus_string, 
        SUM(group_count) AS total_count
    FROM group_counts
    GROUP BY skus_string
),
proportional_sample_size AS (
    SELECT 
        g.skus_string, 
        g.transaction_group, 
        g.transaction_range,
        g.group_count, 
        t.total_count,
        ROUND(g.group_count * 100.0 / t.total_count) AS stratified_sample_size  -- Adjust sample size to 100
    FROM group_counts g
    JOIN total_counts t ON g.skus_string = t.skus_string
),
numbered_stores AS (
    SELECT 
        c.skus_string, 
        c.Site, 
        c.transaction_group,
        c.transaction_range,
        c.weekly_average_baskets,
        p.stratified_sample_size,
        ROW_NUMBER() OVER (PARTITION BY c.skus_string, c.transaction_group ORDER BY RAND()) AS row_num
    FROM classified_stores c
    JOIN proportional_sample_size p 
        ON c.skus_string = p.skus_string 
        AND c.transaction_group = p.transaction_group
),
stratified_sample AS (
    SELECT 
        skus_string, 
        Site, 
        transaction_group,
        transaction_range,
        weekly_average_baskets
    FROM numbered_stores
    WHERE row_num <= stratified_sample_size
),
statistics AS (
    SELECT
        skus_string,
        transaction_group,
        transaction_range,
        AVG(weekly_average_baskets) AS mean_transactions,
        STDDEV(weekly_average_baskets) AS stddev_transactions,
        COUNT(*) AS sample_size -- Added sample size for each group
    FROM stratified_sample
    GROUP BY skus_string, transaction_group, transaction_range
),
sample_size_calculation AS (
    SELECT
        skus_string,
        transaction_group,
        transaction_range,
        stddev_transactions,
        1.96 AS z_alpha,  -- Z-value for 95% confidence
        0.84 AS z_beta,   -- Z-value for 80% power
        mean_transactions,
        0.1 * mean_transactions AS effect_size,  -- Assuming a 10% effect size based on the mean transactions
        sample_size,
        -- Calculate sample size using the simplified formula
        POWER((1.96 + 0.84) * stddev_transactions / (0.1 * mean_transactions), 2) AS calculated_sample_size_per_group
    FROM statistics
),
control_store_recommendation AS (
    SELECT 
        sample_size_calculation.skus_string, 
        sample_size_calculation.transaction_group,
        sample_size_calculation.transaction_range,
        sample_size AS current_sample_size,
        CEIL(calculated_sample_size_per_group) AS recommended_control_stores,
        CEIL(calculated_sample_size_per_group) AS recommended_test_stores,  -- Assuming 1:1 ratio
        group_count AS total_group_count,  -- Total number of stores in each group
        ROUND(group_count * 100.0 / total_count) AS proportion_in_population,  -- Proportion of each group in the population
        mean_transactions,  -- Mean number of transactions
        stddev_transactions  -- Standard deviation of transactions
    FROM sample_size_calculation
    JOIN proportional_sample_size p 
        ON sample_size_calculation.skus_string = p.skus_string 
        AND sample_size_calculation.transaction_group = p.transaction_group
)

SELECT * 
FROM control_store_recommendation
ORDER BY skus_string, 
    CASE 
        WHEN transaction_group = 'Low' THEN 1 
        WHEN transaction_group = 'Medium' THEN 2 
        ELSE 3 
    END;


"""
).result()
check_df = check.to_dataframe()
check_df


,skus_string,transaction_group,transaction_range,current_sample_size,recommended_control_stores,recommended_test_stores,total_group_count,proportion_in_population,mean_transactions,stddev_transactions
0,"175568,175569,175570,175571,175592,175593,236845,254550,254551,254623,254626,285093,285094,285097,392395",Low,1 - 20,2,55.0,55.0,20,2.0,14.628458,3.845767
1,"175568,175569,175570,175571,175592,175593,236845,254550,254551,254623,254626,285093,285094,285097,392395",Medium,21 - 60,15,23.0,23.0,153,15.0,44.994203,7.634789
2,"175568,175569,175570,175571,175592,175593,236845,254550,254551,254623,254626,285093,285094,285097,392395",High,61+,83,213.0,213.0,840,83.0,138.960998,72.365257
3,"331595,694123",Low,1 - 20,87,250.0,250.0,878,87.0,7.943225,4.485142
4,"331595,694123",Medium,21 - 60,12,66.0,66.0,118,12.0,28.833992,8.334813
5,"331595,694123",High,61+,2,1.0,1.0,18,2.0,20.456522,0.584132


In [11]:
check = client.query(
"""     
SELECT *
FROM `gcp-wow-cart-data-prod-d710.cdm.dim_cartology_campaigns`
WHERE opportunity_owner='Alex Moses' 
AND media_start_date >= "2024-08-01"
"""
).result()
check_df = check.to_dataframe()
check_df


,quote_name,booking_number,business_unit,quote_status,quote_sku,net_amount,opportunity_owner,account,line_name,media,media_location,media_type,media_start_date,media_end_date,campaign_start_date,campaign_end_date,original_campaign_start_date,original_campaign_end_date,campaign_duration_weeks,store_list,quoteline_sku,revenue_sku,quantity,media_spend,duration_weeks,opportunity_name,campaign_objective,event_name,placement,online_department,brand,sku_item_name,quoteline_category,commercial_category,cnz_category_patch,line_name_all,original_booking_number,load_dttm,compliance_media_type
0,WOW20014104,WOW20014104,Supermarket,Stage 100%,None,300650.0,Alex Moses,Woolworths Own Label,CI-10182231,Search Web Banner - Beef,Online,Results Page Display,2024-09-04,2024-09-17,2024-09-04,2024-10-01,2024-09-04,2024-10-01,4,None,"331595,694123",331595,1.0,2750.0,2,Macro BOLT Q1 FY25,Acquire New Customers,None,None,None,Macro,None,None,None,None,"CI-10182231,",None,2024-07-29 03:02:33.768852,None
1,WOW20014104,WOW20014104,Supermarket,Stage 100%,None,300650.0,Alex Moses,Woolworths Own Label,CI-10182248,Category App Banner 1 - Poultry,Online,Results Page Display,2024-09-25,2024-10-01,2024-09-04,2024-10-01,2024-09-04,2024-10-01,4,None,"331595,694123",331595,1.0,2000.0,1,Macro BOLT Q1 FY25,Acquire New Customers,None,None,None,Macro,None,None,None,None,"CI-10182248,",None,2024-07-29 03:02:33.768852,None
2,WOW20014104,WOW20014104,Supermarket,Stage 100%,None,300650.0,Alex Moses,Woolworths Own Label,CI-10182245,Category App Banner 2 - Vegetables,Online,Results Page Display,2024-09-04,2024-09-17,2024-09-04,2024-10-01,2024-09-04,2024-10-01,4,None,"694123,331595",331595,1.0,2750.0,2,Macro BOLT Q1 FY25,Acquire New Customers,None,None,None,Macro,None,None,None,None,"CI-10182245,",None,2024-07-29 03:02:33.768852,None
3,WOW20014104,WOW20014104,Supermarket,Stage 100%,None,300650.0,Alex Moses,Woolworths Own Label,CI-10182244,Search App Banner 2 - Vegetables,Online,Results Page Display,2024-09-04,2024-09-17,2024-09-04,2024-10-01,2024-09-04,2024-10-01,4,None,"331595,694123",331595,1.0,2750.0,2,Macro BOLT Q1 FY25,Acquire New Customers,None,None,None,Macro,None,None,None,None,"CI-10182244,",None,2024-07-29 03:02:33.768852,None
4,WOW20014104,WOW20014104,Supermarket,Stage 100%,None,300650.0,Alex Moses,Woolworths Own Label,CI-10182224,Results Page Card Package - Poultry,None,Results Page Display,2024-09-18,2024-10-01,2024-09-04,2024-10-01,2024-09-04,2024-10-01,4,None,"694123,331595",331595,1.0,0.0,2,Macro BOLT Q1 FY25,Acquire New Customers,None,None,None,Macro,None,None,None,None,"CI-10182224,",None,2024-07-29 03:02:33.768852,None
5,WOW20014104,WOW20014104,Supermarket,Stage 100%,None,300650.0,Alex Moses,Woolworths Own Label,CI-10182241,Search Web Content Card - Poultry,Online,Results Page Display,2024-09-18,2024-10-01,2024-09-04,2024-10-01,2024-09-04,2024-10-01,4,None,"694123,331595",331595,1.0,4000.0,2,Macro BOLT Q1 FY25,Acquire New Customers,None,None,None,Macro,None,None,None,None,"CI-10182241,",None,2024-07-29 03:02:33.768852,None
6,WOW20014104,WOW20014104,Supermarket,Stage 100%,None,300650.0,Alex Moses,Woolworths Own Label,CI-10182250,Category Web Banner - Poultry,Online,Results Page Display,2024-09-25,2024-10-01,2024-09-04,2024-10-01,2024-09-04,2024-10-01,4,None,"331595,694123",331595,1.0,2000.0,1,Macro BOLT Q1 FY25,Acquire New Customers,None,None,None,Macro,None,None,None,None,"CI-10182250,",None,2024-07-29 03:02:33.768852,None
7,WOW20014104,WOW20014104,Supermarket,Stage 100%,None,300650.0,Alex Moses,Woolworths Own Label,CI-10182237,Search Web Content Card - Seasoning & Spices,Online,Results Page Display,2024-09-18,2024-10-01,2024-09-04,2024-10-01,2024-09-04,2024-10-01,4,None,"331595,694123",331595,1.0,3250.0,2,Macro BOLT Q1 FY25,Acquire New Customers,None,None,None,Macro,None,None,None,None,"CI-10182237,",None,2024-07-29 03:02:33.768852,None
8,WOW20014104,WOW20014104,Supermarket,Stage 100%,None,300650.0,Alex Moses,Woolworths Own Label,CI-10186278,Category App Ba